# Importing libraries

In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.llms import OpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.prompts import PromptTemplate
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import HumanMessage,AIMessage,SystemMessage
from langchain.chat_models import ChatOpenAI
import random
import pdf2image 
from PIL import Image
import google.generativeai as genai
from langchain.prompts import PromptTemplate
from pdf2image import convert_from_path
from langchain_community.vectorstores import Chroma,FAISS
import json
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory,ConversationSummaryMemory
from langchain.chains.question_answering import load_qa_chain
import time
# from openai import OpenAI
from langchain_chroma import Chroma
from langchain.chains import ConversationChain
import random


# API KEYS 


In [2]:
open_ai_key="OPEN_AI_API_KEY_HERE"
os.environ["OPENAI_API_KEY"] = open_ai_key
genai.configure(api_key='GEMINI_API_KEY_HERE')
os.environ['GROQ_API_KEY']='GROQ_API_KEY_HERE'

# Model Intialization 

In [3]:
model = genai.GenerativeModel("gemini-1.5-flash-latest")
chat_llm = ChatOpenAI(temperature=0.5)
chat = ChatGroq(temperature=0.4, model_name="Mixtral-8x7b-32768")
Chroma_db = Chroma(embedding_function=OpenAIEmbeddings())
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=200,
    length_function=len,
)
chat2=ChatGroq(api_key='GROQ_API_KEY_HERE',temperature=0.6,model='Llama3-70b-8192')
chat3=ChatGroq(api_key='GROQ_API_KEY_HERE',model='Llama3-8b-8192',temperature=0.6)
models=[chat,chat2,chat3]

/Users/mac/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


# PDF TO TEXT 

In [4]:
def pdf_to_text(pdf_path):
    pages=[]
    images= convert_from_path(pdf_path)
    for i in range(len(images)):
        images[i].save('page.jpg')
        img=Image.open('page.jpg')
        res=model.generate_content(['''
    Act as a image to text converter.if the image contains text then extract text from it and add some value to make
    it more meaningful for further processing.Also if image contains diagrams,flowcharts or any thing other than text
    then generate text giving a detailed description about that diagram.
    All the informational should be meaningful and in technical language.
    ''',img])
        try:
            print("page"+str(i+1)+"done")
            pages.append(res.text)
        except:
            pass
    return pages

# TEXT TO DOCUMENT

In [5]:
from dataclasses import dataclass
@dataclass
class Document:
    page_content: str
    metadata: dict
def text_to_doc(pages):
    documents=[]
    j=1
    for i in pages:
        documents.append(Document(page_content=i, metadata={'source':'source', 'page': j}))
        j+=1
    return documents

# Text Splitter

In [6]:
def split_text(docs):
    texts=text_splitter.split_documents(docs)
    return texts

# ADD TO DATABASE

In [7]:
def add_to_db(docs):
    db=Chroma()
    faiss_db = Chroma.from_documents(docs,OpenAIEmbeddings())
    return faiss_db

# MAIN TEXT EXTRACTER

In [8]:
def extract_text(pdf_path):
    pages=pdf_to_text(pdf_path)
    docs=text_to_doc(pages)
    faiss_db=None
    return faiss_db,docs,pages

In [9]:
faiss_db,docs,pages=extract_text('./cell_guide.pdf')

page1done
page2done


In [61]:
ans=start_quiz2('hard',docs,pages,0)

memory Initialed


In [10]:
def convert_json(que,model):
    ind=que.index('[')
    que=que[ind:]
    i=0
    que = que.replace('\n', '')
    try:
        que=que.replace('\\','')
    except:
         pass
    json_list=json.loads(que)
    for i in range(len(json_list)):
        (json_list[i])['id']=i
        (json_list[i])['answered']=0
    return json_list

In [158]:
type(convert_json(que,'mixtral'))

list

# Main Model 

In [11]:
def start_quiz(pdf_path,level):
    pdf_to_text(pdf_path)
    split_text()
    add_to_db()
    dic={}
    if 'flowmessages' not in dic.keys():
        prompt=PromptTemplate(template='''
    Greetings! For this task, you will act as a quiz bot. 
    Your goal is to ask a single question based on the provided data and generate 
    four unique options. Three of these options should be incorrect, while one should 
    be correct. Additionally, provide an explanation for why the correct option is 
    indeed correct, using your intelligence to craft a descriptive explanation.

    Ensure that the questions are relevant to the given text and also 
    dont ask any question related to any image technique or any image 

    Please provide the questions in JSON format, structured as follows:
      "id":"unique integer id"  
      "question": "The question you're asking",
      "option1": "Incorrect option 1",
      "option2": "Incorrect option 2",
      "option3": "Incorrect option 3",
      "option4": "Correct option"
      "correct_option": "D",
      "answered":integer:0,
      "chosen":"",
      "summary": "Explanation for why the correct option is correct"
    Strive to develop questions and options quickly, within 2-3 seconds.
    questions asked should be with a difficuly level of {diff_level}
    You can also use history content to make more questions but remember questions and 
    options should be unique every time
    ''',input_variables=['diff_level'])

        dic['flowmessages']=[SystemMessage(content=prompt.format(diff_level=level))]
        for i in range(3):
            ind=random.randint(0,len(texts)-1)
            ind2=random.randint(0,len(texts[ind])-1)
            con=texts[ind][ind2]
#             print(con)
            dic['flowmessages'].append(HumanMessage(content=con))
            ans=chat_llm(dic['flowmessages'])
            dic['flowmessages'].append(AIMessage(content=ans.content))
            res=json.loads(ans.content)
#             print(type(ans.content))
            questions.append(res)
            print('Q'+str(i+1)+')'+ans.content)
            for i in range(100):
                print('-',end='')
            print()
#         time.sleep(3)

In [12]:
memory=None
def start_quiz3(level,docs,pages,memo):
    global memory
    template='''
    Greetings! For this task, you will act as a quiz bot. 
    Your goal is to ask generate only 9 different question based on the provided data and generate 
    four unique options. Three of these options should be incorrect, while one should 
    be correct. Additionally, provide an explanation for why the correct option is 
    indeed correct, using your intelligence to craft a descriptive explanation.
    don,t write anything other than questions
    remember:-remove any newline character
    Ensure that the questions are relevant to the given text and also 
    dont ask any question related to any image technique or any image 

    Please provide the questions in JSON format, structured as follows:
      "id":"unique integer id starting from 0,1,2 so-on"  
      "question": "The question you're asking",
      "option1": "Incorrect option 1",
      "option2": "Incorrect option 2",
      "option3": "Incorrect option 3",
      "option4": "Correct option"
      "correct_option": "D",
      "answered":"integer 0 only",
      "chosen":"",
      "summary": "Explanation for why the correct option is correct"
    Strive to develop questions and options quickly, within 2-3 seconds.
    difficulty level of the questions should be {human_input}.
    You can also use history content to make more questions but remember questions and 
    options should be unique every time.You are required to develop a Unique question each time 
    use memory to saw previous questions and develop completely different questions
    
        {chat_history}
Content for forming questions: {context}
Chatbot:""
'''
    prompt = PromptTemplate(input_variables=["chat_history", "human_input", "context"], template=template)
    if (memo==0):
        print("memory Initialed")
        memory = ConversationBufferMemory(memory_key="chat_history", input_key="human_input")
    else:
        print("using previous memory")
    chain = load_qa_chain(chat, chain_type="stuff", memory=memory, prompt=prompt)
#     db,docs,pages=extract_text(docs)
    if(len(pages)>5):
        no=random.randint(0,len(pages)-5)
        x=chain({"input_documents":docs[no:no+5], "human_input":level}, return_only_outputs=True)
    else:
        x=chain({"input_documents":docs, "human_input":level}, return_only_outputs=True)
    return x['output_text']

In [13]:
conversation=None
def start_quiz2(level,docs,pages,memo):
#     chat_llm= ChatGroq(temperature=0.6, model_name="Mixtral-8x7b-32768")
    global conversation
    prompt1=PromptTemplate(template='''
    SYSTEM:Greetings! For this task, you will act as a quiz bot. 
    Your goal is to ask generate only 9 different question based on the provided data and generate 
    four unique options. Three of these options should be incorrect, while one should 
    be correct. Additionally, provide an explanation for why the correct option is 
    indeed correct, using your intelligence to craft a descriptive explanation.
    don,t write anything other than questions
    remember:-remove any newline character
    Ensure that the questions are relevant to the given text and also 
    dont ask any question related to any image technique or any image 

    Please provide the questions in JSON format, structured as follows:
      "id":"unique integer id starting from 0,1,2 so-on"  
      "question": "The question you're asking",
      "option1": "Incorrect option 1",
      "option2": "Incorrect option 2",
      "option3": "Incorrect option 3",
      "option4": "Correct option"
      "correct_option": "D",
      "answered":"integer 0 only",
      "chosen":"",
      "summary": "Explanation for why the correct option is correct"
    Strive to develop questions and options quickly, within 2-3 seconds.
    difficulty level of the questions should be {level}.
    ''',input_variables=['level'])
    PROMPT=PromptTemplate(template=prompt1.format(level=level)+
    '''You can also use history content to make more questions but remember questions and 
    options should be unique every time.You are required to develop a Unique question each time 
    use memory to saw previous questions and develop completely different questions
    Already Generated Questions:{history}
    Content for forming new questions: {input}
    AI:""
    ''',input_variables=["history",'input'])
    if(memo==0):
        conversation = ConversationChain(
        prompt=PROMPT,
        llm=chat,
        verbose=True,
        memory=ConversationBufferMemory(k=2),
#         memory=ConversationSummaryMemory(llm=OpenAI(), max_token_limit=2000)
        )
        print("memory Initialized")
    if(len(pages)>5):
        no=random.randint(0,len(pages)-5)
        questions=conversation.predict(input=pages[no:no+5])
    else:
        questions=conversation.predict(input=pages)
    return questions

In [45]:
random.randrange(0,3)

0

In [31]:
db,docs,page=extract_text('./cell_guide.pdf')

page1done
page2done


In [129]:
%%time
# que=start_quiz2('hard',pages,docs,0)
que=start_quiz2('hard',docs,pages,1)



> Entering new ConversationChain chain...
Prompt after formatting:

    SYSTEM:Greetings! For this task, you will act as a quiz bot. 
    Your goal is to ask generate only 9 different question based on the provided data and generate 
    four unique options. Three of these options should be incorrect, while one should 
    be correct. Additionally, provide an explanation for why the correct option is 
    indeed correct, using your intelligence to craft a descriptive explanation.
    don,t write anything other than questions
    remember:-remove any newline character
    Ensure that the questions are relevant to the given text and also 
    dont ask any question related to any image technique or any image 

    Please provide the questions in JSON format, structured as follows:
      "id":"unique integer id starting from 0,1,2 so-on"  
      "question": "The question you're asking",
      "option1": "Incorrect option 1",
      "option2": "Incorrect option 2",
      "option3": "Incorr


> Finished chain.
CPU times: user 38.2 ms, sys: 15 ms, total: 53.2 ms
Wall time: 5.25 s


# Part 2 of APP

# paragraph question Answering for University Exam prep

In [14]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
# from fastapi import FastAPI
# from langserve import add_routes
import pdf2image 
from PIL import Image
import google.generativeai as genai

In [15]:
os.environ['OPENAI_API_KEY']='OPEN_AI_API_KEY'
os.environ['LANGCHAIN_API_KEY']='LANGCHAIN_API_KEY_HERE'
os.environ['LANGCHAIN_TRACING_V2']='true'
genai.configure(api_key='GEMINI_API_KEY_HERE')

In [16]:
prompt1=ChatPromptTemplate.from_messages(
[
    ("system",'''Your task is to act as 
    an Examiner, generate a single {diff_level} question based on 
    provided data. The user will then answer these 
    questions.
    geneate relevant questions and the Student(whose is going to answer 
    that question) cannot see the provided data
    donot generate questions like from the provided content answer
    these.
    also generate a unique question everytime'''),
    ("user",'Content:{content}<previous questions>{previous_questions}</previous_questions>')
]
)
prompt2=ChatPromptTemplate.from_messages(
[
    ('system','''You have to act as an Exam papers Evaluator.
     you are Provided with a <question>{question}</question>and the content
     from where the question is generated:<content>{content}</content>
     
     Please provide the questions in this format, structured as follows:
     <Structure>
"Missing points":""
"Bluff Points";""
"Percentage Match":""
"Improved Answer":""
</Structure>
     based on the question and content ou will evaluate the answers based on the following criteria:

Missing Points: Identify the points or keywords that are missing in the student's answer and suggest additions to improve the answer.
Bluff Points: Identify points in the answer that are not relevant to the question asked.
Percentage Match: Calculate the percentage of the answer that is correct based on the question.
in the end also provide a better answer ,you can your knowledge
also to make a better and improved answerlike this:-
Improved answer:also provide a improved answer of around 100 words
''')
    ,
    ('user','ANSWER:<answer>{answer}</answer>')
]
)

In [17]:
para_llm=ChatOpenAI(temperature=0.5)
# model = genai.GenerativeModel('gemini-pro-vision')
output_parser=StrOutputParser()


In [18]:
chain1=LLMChain(llm=para_llm,prompt=prompt1,output_key='question',verbose=True)
chain2=LLMChain(llm=para_llm,prompt=prompt2,verbose=True,output_key='result')

In [19]:
def text_from_image(image):
    image.save('abc.jpg')
    img=Image.open('abc.jpg')
    res=model.generate_content(['''
             Your task is to extract important text from pictures uploaded by users. 
             The pictures may contain flowcharts, diagrams, or any other relevant educational content. 
             The extracted text should be accurate and relevant for further processing.
            
            ''',img])
    return res

In [20]:
input_text='''
Players from the seven States ,
Telangana, Andhra Pradesh, Tamil Nadu, 
Karnataka, Kerala, \nMaharashtra and Odisha, besides 
some internationals will be in action.  \n• For the first 
time in India, Carrom Live Platform will be paperless with 
the s cores to be entered \nthrough mobiles, live scores and 
online statistics, besides live streaming.  \nTelangana 
Launches State Robotics Framework  \n• Telangana announced 
Robotics Framework   which would encompass creating an 
ecosystem for \nrobotics that champions inno vation, 
entrepreneurship, and research and development, and set
\nup the Telangana Robotics Innovation Centre as a (TRIC) 
\n• It is developed by Telangana’s Department of Information
Technology’s Emerging Technologies \nWing, in conjunction 
with the All India Robo tics Association'
'''

In [97]:
x=chain1.invoke({'diff_level':'easy','content':input_text})



> Entering new LLMChain chain...
Prompt after formatting:
System: Your task is to act as 
    an Examiner, generate a single easy question based on 
    provided data. The user will then answer these 
    questions.
    geneate relevant questions and the Student(whose is going to answer 
    that question) cannot see the provided data
    donot generate questions like from the provided content answer
    these.
    also generate a unique question everytime
Human: Content:
Players from the seven States ,
Telangana, Andhra Pradesh, Tamil Nadu, 
Karnataka, Kerala, 
Maharashtra and Odisha, besides 
some internationals will be in action.  
• For the first 
time in India, Carrom Live Platform will be paperless with 
the s cores to be entered 
through mobiles, live scores and 
online statistics, besides live streaming.  
Telangana 
Launches State Robotics Framework  
• Telangana announced 
Robotics Framework   which would encompass creating an 
ecosystem for 
robotics that champions inno va

In [99]:
ans='''
Telangana announced the launch of a State Robotics Framework 
to promote innovation, entrepreneurship, and research 
and development in the field of robotics. This framework 
includes creating an ecosystem for robotics and setting 
up the Telangana Robotics Innovation Centre (TRIC). 
It is developed by Telangana’s Department of Information 
Technology’s Emerging Technologies Wing, in 
conjunction with the All India Robotics Association.
'''

In [100]:
y=chain2.invoke({'content':input_text,'question':x,'answer':ans})



> Entering new LLMChain chain...
Prompt after formatting:
System: You have to act as an Exam papers Evaluator.
     you are Provided with a {'diff_level': 'easy', 'content': "\nPlayers from the seven States ,\nTelangana, Andhra Pradesh, Tamil Nadu, \nKarnataka, Kerala, \nMaharashtra and Odisha, besides \nsome internationals will be in action.  \n• For the first \ntime in India, Carrom Live Platform will be paperless with \nthe s cores to be entered \nthrough mobiles, live scores and \nonline statistics, besides live streaming.  \nTelangana \nLaunches State Robotics Framework  \n• Telangana announced \nRobotics Framework   which would encompass creating an \necosystem for \nrobotics that champions inno vation, \nentrepreneurship, and research and development, and set\n\nup the Telangana Robotics Innovation Centre as a (TRIC) \n\n• It is developed by Telangana’s Department of Information\nTechnology’s Emerging Technologies \nWing, in conjunction \nwith the All India Robo tics Associati

In [101]:
print(y['result'])

Missing Points:
1. The student's answer is missing the mention of Telangana as the Indian state that recently launched the Robotics Framework.
2. The student did not mention that the framework aims to promote innovation, entrepreneurship, and research and development in robotics.
3. The student did not include the detail about the Telangana Robotics Innovation Centre (TRIC) being set up as part of this initiative.

Bluff Points:
There are no bluff points in the student's answer.

Percentage Match:
The student's answer covers about 85% of the information provided in the question.

Improved answer:
Telangana recently launched a State Robotics Framework to foster innovation, entrepreneurship, and research and development in the field of robotics. This framework aims to create an ecosystem that supports robotics and has led to the establishment of the Telangana Robotics Innovation Centre (TRIC). Developed by Telangana’s Department of Information Technology’s Emerging Technologies Wing in c

In [22]:
para_content =None
page_numbers=[]
para_questions=[]
def gen_ques(pages,level,para_questions):
    global para_content
    prque=''
    page_numbers.clear()
#     texts=split_text(docs)
    for i in range(0,3):
        if len(pages)>2:
            ind = random.randint(0, len(pages) - 1)
            page_numbers.append(ind)
            para_content = pages[ind]
            x = chain1.invoke({'diff_level': level, 'content':para_content,'previous_questions':prque})
        else:
            x = chain1.invoke({'diff_level': level, 'content':pages,'previous_questions':prque})
        prque=prque+x['question']+'\n'
        para_questions.append({'id':i,'answered':0,'question':x['question'],'chosen':''})
        print(page_numbers)
    return para_questions

In [23]:
def check_ans(pages,user_ans):
    result=[]
# user_ans=json.loads(user_ans)
    for i in range(3):
        que=(user_ans[i])['question']
        ans=(user_ans[i])['chosen']
        if(len(pages)>2):
            output=(chain2.invoke({'content':pages[page_numbers[i]],'question':que,'answer':ans}))['result']
        else:
             output=(chain2.invoke({'content':pages,'question':que,'answer':ans}))['result']
        print(output)
        output=output.replace('\n','<br></br>')
        output='Q. '+que+'<br></br>'+output
        result.append({'result':output})
    return result

In [109]:
gen_ques(pages,'hard',para_questions)



> Entering new LLMChain chain...
Prompt after formatting:
System: Your task is to act as 
    an Examiner, generate a single hard question based on 
    provided data. The user will then answer these 
    questions.
    geneate relevant questions and the Student(whose is going to answer 
    that question) cannot see the provided data
    donot generate questions like from the provided content answer
    these.
    also generate a unique question everytime
Human: Content:The image contains a list of J.P. Morgan Chase Bank branches:

* J.P. Morgan Chase Bank - Hong Kong Branch, Chater House, 8 Connaught Road Central, Hong Kong
* J.P. Morgan Chase Bank - Singapore Branch, 168 Robinson Road, Singapore
* J.P. Morgan Services Argentina S.R.L., Av. Belgrano 955, 1st Floor, C.A.B.A, Argentina
* J.P. Morgan Services India Private Limited, Prism Towers, Levels 9 to 11, Mindpace, Link Road, Goregaon (West), Mumbai 400 104, India

©2024 JPMorgan Chase & Co. JPMorgan Chase is an equal opportunit

[{'id': 0,
  'answered': 0,
  'question': 'What are the addresses of J.P. Morgan Chase Bank branches in Hong Kong, Singapore, Argentina, and India?',
  'chosen': ''},
 {'id': 1,
  'answered': 0,
  'question': 'What are the specific consequences outlined in the document for participants who do not comply with the terms and conditions of the Hackathon organized by JPMC?',
  'chosen': ''},
 {'id': 2,
  'answered': 0,
  'question': 'Question: How does JPMC ensure the fairness and impartiality of the judging process in the Code for Good Hackathon 2024, considering the potential impact of prize awards on participants from different regions/countries?',
  'chosen': ''}]

In [28]:
para_questions

[{'id': 0,
  'answered': 0,
  'question': 'Question: In what ways does J.P. Morgan Chase demonstrate its commitment to diversity and inclusion in its employment practices?',
  'chosen': ''},
 {'id': 1,
  'answered': 0,
  'question': 'Question: In the context of the Hackathon, why is it important to clarify that participation does not automatically result in an employment relationship with JPMC or the non-profit organization?',
  'chosen': ''},
 {'id': 2,
  'answered': 0,
  'question': 'Question: In the context of the Hackathon, what are the key points highlighted regarding jurisdiction for disputes in different countries and the relationship between participation in the event and employment with JPMC or the non-profit?',
  'chosen': ''}]

In [110]:
page_numbers

[3, 1, 0]

In [29]:
# gen_ques('./cell_guide.pdf','medium')
print(check_ans(que,'hi hello bye'))



> Entering new LLMChain chain...
Prompt after formatting:
System: You have to act as an Exam papers Evaluator.
     you are Provided with a What are some key skills expected from participants of "Code for Good" 2024 event organized by JPMorgan Chase? and the content
     from where the question is generated:The document is a call for participation in "Code for Good" 2024. The event is organized by JPMorgan Chase and is aimed at bringing together tech talent to address social problems. The event will take place in Bengaluru, Hyderabad and Mumbai.

The event is open to 2025 batch graduates with a Bachelor's degree in Computer Science, Information Science or Information Technology. Participants should have a minimum aggregate of 8.5 CGPA until the date of engineering course completion.  

The event will require participants to apply online within the given deadline. Selected participants will be notified via email with final event details. Participants will be required to bring their la

# Part 3 User Asking Questions

In [95]:
# Initialize global variables
vector_db = None
docs = None
level = None
para_que = None
pages=None
token_user={}
Multi_quiz_questions=None
user1={}
user2={}
image_desc=None
para_pages=None
quiz_pages=None
chatbot_docs=None

# Flask Server

In [97]:
import json
from flask import Flask, request,send_from_directory
from flask_cors import CORS
token_user={}
app = Flask(__name__)
CORS(app)
Multi_quiz_questions=None
# Routes for single player quiz
@app.route('/extractContent', methods=['POST'])
def extractContent():
    link = request.json.get('link')
    global vector_db, docs, pages
    vector_db, docs, pages = extract_data(link)
    return {"status": "ok"}

@app.route('/startQuiz', methods=['POST'])
def startQuiz():
    global level, docs, quiz_pages
    var = request.json.get('var')
#     print(var)
    if(var==True):
        link = request.json.get('link')
        level = request.json.get('level')
        db,docs,quiz_pages=extract_data(link)
        print(quiz_pages)
        questions = start_quiz2(level, docs, quiz_pages,0)
#         que = questions.replace('\n', '')
#         data_json = json.loads(que)
        data_json=convert_json(questions,'mixtral')
        return {"questions": data_json}
    else:
        level = request.json.get('level')
        questions = start_quiz2(level, docs, quiz_pages,1)
#         que = questions.replace('\n', '')
#         data_json = json.loads(que)
        data_json=convert_json(questions,'mixtral')
        return {"questions": data_json}

# Routes for two player quiz
@app.route('/generateToken', methods=['POST'])
def generate_token():
    global level
#     level = request.json.get('level')
    username = request.json.get('username')
    token = gen_tok(username)
    return {'token':token}

@app.route('/start2player', methods=['POST'])
def start2player():
    global Multi_quiz_questions,token_user
#     global level, docs
    chstatus=request.json.get('chStatus')
    var=request.json.get('status')
    if(var==400 or var==404):
        username = request.json.get('username')
        if username in token_user.keys():
            if token_user[username][2]=='YES':
                return {"status":"200"}
    if(chstatus):
        link=request.json.get('link')
        level=request.json.get('level')
        username = request.json.get('username')
        db,docs,pages=extract_data(link)
        Multi_quiz_questions = start_quiz2(level, docs, pages,0)
#         que = Multi_quiz_questions.replace('\n', '')
#         data_json = json.loads(que)
#         above 2 for mixtral model
        data_json=convert_json(Multi_quiz_questions,'mixtral')
#         print(username)
#         print(token_user)
        global user1,user2,marks1,marks2
        user1={}
        user2={}
        token_user[username][1] = 'YES'
       
        return {"questions":data_json}
    return {"status": "404"}

@app.route('/verifyToken', methods=['POST'])
def verifyToken():
    global Multi_quiz_questions
    token = request.json.get('token')
    print(token_user)
    for user, data in token_user.items():
        if data[0] == token:
            (token_user[user])[2]='YES'
            while((token_user[user])[1]=='NO'):
                pass
            data_json=convert_json(Multi_quiz_questions,'mixtral')
            return {"questions":data_json,"status":"200"}
    return {"status":"404"}
@app.route('/user1submit', methods=['POST'])
def user1submit():
    global user1
    user1['username']=request.json.get('username')
    user1['marks']=request.json.get('marks')
    user1['wrong']=request.json.get('wrong')
    return {'winner':Identify_winner(),'marks':user2['marks'],'wrong':user2['wrong']}
@app.route('/user2submit', methods=['POST'])
def user2submit():
    global user2
    user2['username']=request.json.get('username')
    user2['marks']=request.json.get('marks')
    user2['wrong']=request.json.get('wrong')
    return {'winner':Identify_winner(),'marks':user1['marks'],'wrong':user1['wrong']}
# Routes for paragraph writing questions
@app.route('/paraQuizques', methods=['POST'])
def para_Quiz_ques():
    global docs, para_que
    level = request.json.get('level')
    para_que = gen_ques(docs, level)
    return {"question": para_que}

@app.route('/paraQuizans', methods=['POST'])
def para_Quiz_Ans():
    global para_que
    ans = request.json.get('answer')
    feedback = check_ans(para_que, ans)
    return {"ans_status": feedback}

# Route for asking queries
@app.route('/askQuery', methods=['POST'])
def askQuery():
    global vector_db
    check = request.json.get('var')
    if check == 0:
        link = request.json.get('link')
        vector_db, docs, pages = extract_data(link)
        vector_db=add_to_db(docs)
        return {"status": "ok"}
    else:
        query = request.json.get('query')
#         print(vector_db)
        answer = query_check(query,vector_db,'local','Technical')
        return {"answer":answer}
@app.route('/websearch', methods=['POST'])
def webSearch():
    query=request.json.get('query')
#     print(query)
    ans=query_check(query,vector_db,'web','Technical')
#     ans=web_Search(query)
    return {'answer':ans}
#     return {'ans':'hello'}
####SSB IMAGE TEST ########
@app.route('/ssbimage', methods=['POST'])
def ssbimage():
    global image_desc
    image_desc=SSB_Image_generator(0)
    return send_from_directory('./', 'main.png')
@app.route('/qualitycheck', methods=['POST'])  
def qualityCheck():
    user_story = request.json.get('story')
    return {"description":SSB_TEST_STORY(image_desc,user_story)}
####Para question answering####
@app.route('/paraque', methods=['POST']) 
def paraque():
    global para_pages
    para_questions=[]
    link = request.json.get('link')
    vector_db, docs, para_pages = extract_data(link)
    que=gen_ques(para_pages,level,para_questions)
#     print(que)
    return {'questions':que}
@app.route('/paraanswer', methods=['POST']) 
def paraAnswer():
    global para_pages
    user_ans=(request.json)['dummy']
    print(user_ans)
    result=check_ans(para_pages,user_ans)
    print(result)
    return {'result':result}
#     return {"status":200}
@app.route('/placement_que', methods=['POST']) 
def placement_que():
    topic=gen_placement_que()
    return {'question':topic}
@app.route('/placement_res', methods=['POST']) 
def placement_res():
    topic=request.json.get('question')
    essay=request.json.get('answer')
    res=check_placement_ans(topic,essay)
    return {'result':res}
if __name__ == '__main__':
    app.run(host="0.0.0.0")

    

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.13:5000
Press CTRL+C to quit
192.168.1.15 - - [03/Jun/2024 16:14:29] "OPTIONS /askQuery HTTP/1.1" 200 -
192.168.1.15 - - [03/Jun/2024 16:14:29] "OPTIONS /askQuery HTTP/1.1" 200 -


DONE1
DONE1
page1done
page1done


[2024-06-03 16:14:42,380] ERROR in app: Exception on /askQuery [POST]
Traceback (most recent call last):
  File "/Users/mac/anaconda3/lib/python3.11/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/anaconda3/lib/python3.11/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/anaconda3/lib/python3.11/site-packages/flask_cors/extension.py", line 176, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
                                                ^^^^^^^^^^^^^^^^^^
  File "/Users/mac/anaconda3/lib/python3.11/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/anaconda3/lib/python3.11/site-packages/flask/app.py", line 1796, in dispatch_req

page2done


...

{'Rating': 1}
"What is HackerEarth?"


192.168.1.15 - - [03/Jun/2024 16:39:29] "POST /askQuery HTTP/1.1" 200 -
192.168.1.15 - - [03/Jun/2024 16:39:40] "OPTIONS /websearch HTTP/1.1" 200 -


{'Rating': 1}
called


> Entering new AgentExecutor chain...

I will search for 'What is HackerEarth?' and write an answer based on the results.
{'tool_name': 'tavily_search_results_json', 'parameters': {'query': 'What is HackerEarth?'}}
[{'url': 'https://en.wikipedia.org/wiki/HackerEarth', 'content': 'HackerEarth is a software company headquartered in San Francisco that provides enterprise software that assists organizations with technical hiring. HackerEarth is used by organizations for technical skill assessments and remote video interviewing .'}, {'url': 'https://www.linkedin.com/company/hackerearth', 'content': 'HackerEarth is the most comprehensive developer assessment software that helps companies to accurately measure the skills of developers during the recruiting process.'}, {'url': 'https://www.hackerearth.com/recruit/assessments/', 'content': 'HackerEarth allows you to customize the difficulty level of your coding assessments in several ways. You can choose from a library of

192.168.1.15 - - [03/Jun/2024 16:39:53] "POST /websearch HTTP/1.1" 200 -


Relevant Documents: 0,1,2,3,4
Cited Documents: 0,1,2,3,4
Answer: HackerEarth is a software company that provides enterprise software to help organisations with technical hiring. It is used by engineering teams to automate their hiring process and select the best fit for their team. HackerEarth allows users to customise the difficulty level of their coding assessments, choose from a library of pre-built questions with varying difficulty levels, or create their own custom questions.
Grounded answer: HackerEarth is a <co: 0>software company</co: 0> that provides <co: 0>enterprise software</co: 0> to help <co: 0,1,4>organisations with technical hiring</co: 0,1,4>. It is <co: 3>used by engineering teams</co: 3> to <co: 4>automate their hiring process</co: 4> and <co: 4>select the best fit for their team</co: 4>. HackerEarth allows users to <co: 2>customise the difficulty level of their coding assessments</co: 2>, <co: 2>choose from a library of pre-built questions with varying difficulty le

# Download Pdf

In [26]:
import requests
def extract_data(link):
    global para_que
    drive_link = link
    file_id = drive_link.split("/")[-2]
    download_link = f"https://drive.google.com/uc?id={file_id}"
    response = requests.get(download_link)
    with open("main.pdf", "wb") as file:
        file.write(response.content)
        
    print('DONE1')
    db,docs,pages=extract_text('./main.pdf')
    return db,docs,pages

In [127]:
db,docs,pages=extract_data('https://drive.google.com/file/d/19fD_lHWqUd89VV7epg83i1rTkTkOyVEv/view?usp=share_link')

DONE1
page1done
page2done
page3done
page4done
page5done
page6done
page7done


In [44]:
SSB_TEST_STORY(image_desc,'hello')

The candidate's response lacks content and does not align with the image provided for the Picture Perception and Discussion Test (PPDT). The story is incomplete and does not reflect the details of the image depicting a group discussion on environmental conservation and climate change impact on Indian monsoons.

Observation Skills: The candidate did not describe the key elements of the image, such as the diverse group of candidates, the setting of the SSB exam room, the discussion topic on environmental conservation, and the map of India with environmental markers.

Imagination: The candidate's story is missing, and therefore, there is a lack of creativity or originality in creating a narrative based on the image.

Writing Style: The response is extremely brief, with only the word "hello" provided. There are no grammar, vocabulary, or sentence structure to evaluate in this context.

Officer Like Qualities (OLQs): The response does not demonstrate any leadership, initiative, effective co

'The candidate\'s response lacks content and does not align with the image provided for the Picture Perception and Discussion Test (PPDT). The story is incomplete and does not reflect the details of the image depicting a group discussion on environmental conservation and climate change impact on Indian monsoons.\n\nObservation Skills: The candidate did not describe the key elements of the image, such as the diverse group of candidates, the setting of the SSB exam room, the discussion topic on environmental conservation, and the map of India with environmental markers.\n\nImagination: The candidate\'s story is missing, and therefore, there is a lack of creativity or originality in creating a narrative based on the image.\n\nWriting Style: The response is extremely brief, with only the word "hello" provided. There are no grammar, vocabulary, or sentence structure to evaluate in this context.\n\nOfficer Like Qualities (OLQs): The response does not demonstrate any leadership, initiative, e

# 2 Player Competetion

In [27]:
import uuid

In [28]:
def gen_tok(username):
    token=str(uuid.uuid1())[0:8]
    token_user[username]=[token,'NO','NO']# 1st No=>1st player 2nd No for 2nd player
    return token

In [29]:
def Identify_winner():
    while(user1=={} or user2=={}):
        pass
    if(user1['marks']>user2['marks']):
        return user1['username']
    else:
        return user2['username']

# Part 4 QA chatbot


In [30]:
from langchain.prompts import PromptTemplate
import os
from langchain_community.vectorstores import Chroma,FAISS
import json
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain.memory import ConversationBufferMemory
import time
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_cohere import CohereRerank
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.agents import AgentExecutor
from langchain_cohere.chat_models import ChatCohere
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate
from langchain.llms import OpenAI

In [31]:
os.environ['GROQ_API_KEY']='GRQO_API_KEY_HERE'
open_ai_key="OPEN_AI_API_KEY_HERE"
os.environ["OPENAI_API_KEY"] = open_ai_key
os.environ['COHERE_API_KEY']='COHERE_API_KEY_HERE'
os.environ['TAVILY_API_KEY']='TAVILY_API_KEY_HRE'

In [32]:
standalone_question_llm=ChatGroq(temperature=0.0, model_name="mixtral-8x7b-32768",api_key='GROQ_API_KEY_HERE')
answering_llm=ChatGroq(temperature=0.6, model_name="mixtral-8x7b-32768",api_key='GROQ_API_KEY_HERE')
compressor = CohereRerank(top_n=4)
query_check_llm=OpenAI(temperature=0.5)
greet_llm=ChatGroq(temperature=0.6, model_name="mixtral-8x7b-32768")

/Users/mac/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [76]:
template1 = """
[INST] 
Given the following conversation and a follow up question, 
rephrase the follow up question to be a standalone question, in its original language, 
that can be used to query a FAISS index. This query will be used to retrieve documents with additional context.

Let me share a couple examples.
If you do not see any chat history, you MUST return the "Follow Up Input" as is:
```
Chat History:
Follow Up Input: How is Lawrence doing?
Standalone Question:
How is Lawrence doing?
```

If this is the second question onwards, you should properly rephrase the question like this:
```
Chat History:
Human: How is Lawrence doing?
AI: 
Lawrence is injured and out for the season.
Follow Up Input: What was his injury?
Standalone Question:
What was Lawrence's injury?
```

Now, with those examples, here is the actual chat history and input question.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:
[your response here]
[/INST] 
"""
prompt2 = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.
<context>{context}</context>
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 

Question: {input}
Kindly write the output in a Technical way""")
query_check_template = '''
Please analyze the following text and provide a rating for the level of inappropriate content.
Rate it on a scale of 1 to 5, where 1 indicates no inappropriate content and 5
indicates highly inappropriate content. Only provide the rating, no need to explain why:

Text: {query}

If you find the text as a greeting statement like "Good morning", "Good evening", "Hi", "Hello",
"How are you", "Bye", "Goodbye", or anything you think is similar, rate them as -1.
give results as:
Rating:rate
'''


In [77]:
prompt1 = PromptTemplate(input_variables=["chat_history", "question"], template=template1)
QA_memory = ConversationBufferMemory(memory_key="chat_history", input_key="question")
chain=prompt1|standalone_question_llm
#----------------------------------------------------------------------------------------------------------
query_check_prompt=PromptTemplate(input_variables=['query'],template=query_check_template)
query_check_chain=query_check_prompt|query_check_llm
#----------------------------------------------------------------------------------------------------------


In [78]:
def query_check(query,faiss_db,search,way='Technical'):
    rating=query_check_chain.invoke({"query":query})
    key, value = rating.split(': ')
    result_dict = {key.strip(): int(value)}
    print(result_dict)
    if(result_dict['Rating']==-1):
        greet_template='''System Message:You are a helpful assistant. Greet people Politely
        user Text:{query}'''
        greet_prompt = PromptTemplate(input_variables=['query'],template=greet_template)
        greet_chain = greet_prompt| greet_llm
        ans=greet_chain.invoke({"query":query})
        return ans.content
    elif(result_dict['Rating']<4):
        if(search=='local'):
            ans=ans_query(query,faiss_db,way)
        elif (search=='web'):
            ans=web_Search(query)
        return ans
    else:
        return "Inappropriate Query"

In [96]:
def ans_query(query,faiss_db,way='Technical'):
        global QA_memory
        main_question=chain.invoke({"question":query,"chat_history":QA_memory}).content
        QA_memory.save_context({'question':query},{'answer':main_question})
        print(main_question)
        retriever=faiss_db.as_retriever() 
        compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor, 
        base_retriever=retriever
        )
        document_chain=create_stuff_documents_chain(answering_llm,prompt2)
        retrieval_chain=create_retrieval_chain(compression_retriever,document_chain)
        ans=retrieval_chain.invoke({'input':main_question})
        return ans['answer']
#         model=OpenAI(temperature=0.5)
#         chain2=prompt2|model|output_parser
#         res=chain2.invoke({'input':main_question})
#         return res

In [80]:
query='what is jp morgan code chase'
ans_query(query,db)

"What is the code for JP Morgan Chase?"


'Based on the provided context, the question seems to be misinterpreted as it is not related to programming code, but rather might be asking for a code or identifier for JP Morgan Chase.\n\nJP Morgan Chase does not have a single code, but they have different codes, identifiers, and abbreviations depending on the context. Some of them include:\n\n1. JPM (New York Stock Exchange Ticker Symbol): JPM is the ticker symbol for JP Morgan Chase on the New York Stock Exchange (NYSE).\n\n2. JPMorgan Chase Bank, N.A.: JPMorgan Chase Bank, N.A. is the legal name of the banking entity, and it can be used as a code or identifier for JP Morgan Chase in various situations.\n\n3. SWIFT/BIC Code: The SWIFT/BIC code for JPMorgan Chase Bank, N.A. varies depending on the location, but an example is CHASUS33.\n\n4. Routing Transit Number (RTN) or ABA Number: JPMorgan Chase Bank, N.A. uses different routing transit numbers (RTNs) or ABA numbers depending on the location and the type of transaction.\n\n5. LEI

In [37]:
def web_Search(query):
    print('called')
    main_question=chain.invoke({"question":query,"chat_history":QA_memory}).content
    internet_search = TavilySearchResults()
    llm = ChatCohere()
    agent = create_cohere_react_agent(
        llm=llm,
        tools=[internet_search],
        prompt=ChatPromptTemplate.from_template("""Answer the following question using the Web Tools 
    Think step by step before providing a detailed answer. 
    I will tip you $1000 if the user finds the answer helpful.
    <question>
    {question}
    </question>
    Answer the Question in around 1 paragraph.
    """),
    )
    agent_executor = AgentExecutor(agent=agent, tools=[internet_search], verbose=True)

    response = agent_executor.invoke({
    "question":main_question,
    })
    try:
        ans=response.get('output')+'\n'+(response.get("citations")[0]).documents[0]['url']+'\n'+(response.get("citations")[1]).documents[0]['url']
    except:
         ans=response.get('output')
    return ans

# SSB IMAGE TEST

In [38]:
import io
import os
import warnings
from IPython.display import display
from PIL import Image
from stability_sdk import client
from stability_sdk.client import generation
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain_groq import ChatGroq
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [39]:
stability_api = client.StabilityInference(
    host='Stable diffusion host url',
    key='stable diffusion api here',
    verbose=False,
)
open_ai_key="OPEN_AI_API_KEY"
os.environ["OPENAI_API_KEY"] = open_ai_key
os.environ['GROQ_API_KEY']='GROQ_API_KEY_HERE'

In [40]:
desc_llm1 = OpenAI(temperature=0.7)
desc_llm2=ChatGroq(temperature=0.7, model_name="mixtral-8x7b-32768")
evalutor_llm1 = ChatOpenAI(streaming=True, callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]), verbose=True, temperature=0.6)
evalutor_llm2= ChatGroq(temperature=0.6, model_name="mixtral-8x7b-32768",streaming=True, callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]), verbose=True)

In [41]:
converstion=None

In [42]:
def SSB_Image_generator(memory):
    global conversation
    template='''SYSTEM:write me a prompt that can be given to model for generating images for Indian SSB exam. 
    Model should give me a detailed description about the image like:- how persons are sitting, 
    what topic they are discussing other elements like enviroment,animals, anything that can be used to check 
    a candidates imagination and leader ship qualities about there means a unique description of image 
    but one thing the model develops output containing 1st line as Create a black and white image .
    Current conversation:
    {history}
    Human: {input}
    AI Assistant:
    '''
    PROMPT = PromptTemplate(input_variables=["history",'input'], template=template)
    if(not memory):
        conversation = ConversationChain(
        prompt=PROMPT,
        llm=desc_llm2,
        verbose=False,
        memory=ConversationBufferMemory(),
        )
        print("memory Initialized")
    description=conversation.predict(input='Write a unique description for me')
    answers = stability_api.generate(
    prompt=description,
    )
    print("generating image")
    for resp in answers:
        for artifact in resp.artifacts:
            if artifact.finish_reason == generation.FILTER:
                warnings.warn(
                    "Your request activated the API's safety filters and could not be processed."
                    "Please modify the prompt and try again.")
            if artifact.type == generation.ARTIFACT_IMAGE:
                img = Image.open(io.BytesIO(artifact.binary))
                img.save('main.png')
    #             display(img)
    image_desc=description[25:]
    return image_desc

In [87]:
def SSB_TEST_STORY(image_desc,story):
    prompt=PromptTemplate(
    template='''
            You are an evaluator at the Services Selection Board (SSB), tasked with evaluating a candidate's response to the Picture Perception and Discussion Test (PPDT). The candidate has provided a brief description of the image and a story based on it. Your task is to evaluate the candidate's performance based on the following criteria:

    Observation Skills: Consider how well the candidate has described the features of the image. Did they capture the key details accurately?
    Imagination: Evaluate the creativity and originality of the candidate's story. Did they demonstrate the ability to create a compelling narrative based on the image?
    Writing Style: Assess the candidate's writing style, including grammar, vocabulary, and sentence structure. Did they communicate their ideas effectively?
    Officer Like Qualities (OLQs): Look for qualities such as leadership, initiative, effective communication, and positive attitude in the candidate's response.
    Provide specific feedback on the candidate's performance, highlighting any grammatical errors, vocabulary usage, or areas for improvement. Finally, rate the candidate's overall performance on a scale of 1 to 10, with 10 being the highest.

    Description of the Image:
    {image_desc}

    will Provide you Candidate's Story''',
    input_variables=["image_desc"]
    )

    system_message_prompt = SystemMessagePromptTemplate(prompt=prompt)
    human_template='''
    <user-story>
    {story}
    </user-story>
    '''
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
    chat_prompt_with_values = chat_prompt.format_prompt(image_desc=image_desc,story=story)
    ans=evalutor_llm1(chat_prompt_with_values.to_messages())
    ans=(ans.content).replace('\n','<br></br>')
    return ans

# PLACEMENT TESTS

In [44]:
placement_ques_model=ChatGroq(temperature=0.6,model_name="mixtral-8x7b-32768")
placement_ans_model=OpenAI(temperature=0.5)

In [83]:
def gen_placement_que():
    prompt=PromptTemplate(
    template='''provide an essay topic that are currently hot 
    and relevant in the tech industry, particularly focusing on themes
    and questions asked by {comp} companies (Facebook, Amazon, Apple, Netflix, Google). T
    he topics should be suitable for evaluating candidates' knowledge, critical thinking, and 
    communication skills. Each topic should be clearly stated and designed to be answered in a
    pproximately 200 words.
    <Example Topics>
    The Ethical Implications of Artificial Intelligence in Social Media Platforms.
    The Role of Cloud Computing in Modern Business.
    The Future of Remote Work and Its Impact on the Tech Industry.
    Data Privacy and User Security in Digital Services.
    </Example Topics>''',
        input_variables=['comp']
    )
    placement_chain=prompt|placement_ques_model|output_parser
    topic=placement_chain.invoke({'comp':'FAANG'})
    ind=topic.index('\n')
    topic=(topic[0:ind])+'''<br></br> Write a 200 words essay on the given topic.<br></br>
    Basis of Evaluation:coherence, relevance to the topic, structure, grammar, vocabulary, and originality of ideas.'''
#     topic=topic.replace('\n','<br></br>')
    return topic

In [84]:
que=gen_placement_que()

In [85]:
que

'"The Impact of Deepfakes on Society and Their Implications for Online Platforms"<br></br> Write a 200 words essay on the given topic.<br></br>\n    Basis of Evaluation:coherence, relevance to the topic, structure, grammar, vocabulary, and originality of ideas.'

In [48]:
ans='''Artificial Intelligence (AI) and Machine Learning (ML) have significantly transformed 
customer experience across various industries. Discuss how AI and ML technologies are being 
utilized to improve customer interactions, personalize services, and predict customer needs. 
Additionally, analyze the potential challenges and ethical considerations that companies should 
address when implementing these technologies. Provide specific examples to support your arguments, 
and suggest ways in which businesses can balance innovation with customer trust and data privacy.'''

In [49]:
def check_placement_ans(que,ans):
    prompt=PromptTemplate(
    template='''
    You are provided with an essay written by a candidate in 
    response to a given topic:
    <topic>
    {topic}
    </topic>
    Evaluate the essay based on the following criteria: 
    coherence, relevance to the topic, structure, grammar, vocabulary, and originality of ideas.
    Provide a detailed evaluation report including scores (out of 10) for each criterion and 
    constructive feedback for improvement.

    <Candidate's Essay>
    {essay}
    </Candidate's Essay>
    ''',input_variables=['essay','topic'])
    placement_chain=prompt|placement_ans_model|output_parser
    res=placement_chain.invoke({'topic':que,'essay':ans})
    res=res.replace('\n','<br></br>')
    return res

In [115]:
check_placement_ans(que,ans)

'<br></br>Coherence:<br></br>Score: 9/10<br></br>Constructive Feedback: The essay is well-structured and flows smoothly from one idea to the next. However, there are a few instances where the transition between paragraphs could be improved for better coherence.<br></br><br></br>Relevance to the topic:<br></br>Score: 10/10<br></br>Constructive Feedback: The essay is highly relevant to the given topic and effectively addresses all the key questions provided. The candidate has demonstrated a strong understanding of the role of AI and ML in enhancing customer experience.<br></br><br></br>Structure:<br></br>Score: 9/10<br></br>Constructive Feedback: The essay has a clear and logical structure, with an introduction, body, and conclusion. However, the introduction could be improved by providing a brief overview of what the essay will cover.<br></br><br></br>Grammar:<br></br>Score: 8/10<br></br>Constructive Feedback: The essay has a few minor grammatical errors, such as missing articles and in